In [19]:
import os
from scipy import misc
import numpy as np
from matplotlib import pyplot as plt
from keras.utils.data_utils import get_file
from six.moves import cPickle
import sys
from keras.utils import np_utils 
import cPickle as pickle
from sklearn.cross_validation import train_test_split
import random

In [20]:
#define the working directory
cwd=os.getcwd()
(cwdtop,tail)=os.path.split(cwd)
patch_dir=os.path.join(cwdtop,'patches')
pickle_dir=os.path.join(cwdtop,'pickle')
if not os.path.exists(pickle_dir):
 os.mkdir(pickle_dir)  


In [21]:
#define a list of used labels
usedclassif = ['consolidation',
 'fibrosis',
 'ground_glass',
 'healthy',
 'micronodules',
 'reticulation']
print (usedclassif)

['consolidation', 'fibrosis', 'ground_glass', 'healthy', 'micronodules', 'reticulation']


In [22]:
#define a dictionary with labels
classif={}
i=0
for f in usedclassif:
    classif[f]=i
    i+=1

In [23]:
print classif

{'micronodules': 4, 'healthy': 3, 'ground_glass': 2, 'consolidation': 0, 'reticulation': 5, 'fibrosis': 1}


In [24]:
# list all directories under patch directory. They are representing the categories

category_list=os.walk( patch_dir).next()[1]


In [25]:
# print what we have as categories
category_list

['air_trapping',
 'bronchial_wall_thickening',
 'bronchiectasis',
 'consolidation',
 'cysts',
 'early_fibrosis',
 'emphysema',
 'fibrosis',
 'ground_glass',
 'healthy',
 'increased_attenuation',
 'macronodules',
 'micronodules',
 'pcp',
 'peripheral_micronodules',
 'reticulation',
 'tuberculosis']

In [26]:
# creating variables
# list for the merged pixel data
dataset_list = []
# list of the label data
label_list = []



# go through all categories
# 
for category in category_list:
    if category in usedclassif:
        category_dir = os.path.join(patch_dir, category)
        print  'the path into the categories is: ', category_dir

        sub_categories_dir_list = (os.listdir(category_dir))
        print 'the sub categories are : ', sub_categories_dir_list

        for subCategory in sub_categories_dir_list:

            subCategory_dir = os.path.join(category_dir, subCategory)
            print  'the path into the sub categories is: '
            print subCategory_dir

            image_files = (os.listdir(subCategory_dir))

            for file in image_files:

                if file.find('.bmp') > 0:

                    # load the .bmp file into array       
                    image = misc.imread(os.path.join(subCategory_dir,file), flatten= 0)
                    # append the array to the dataset list
                    dataset_list.append(image)

                    # created rotated copies of images
                    image90 = np.rot90(image)
                    dataset_list.append(image90)

                    image180 = np.rot90(image90)
                    dataset_list.append(image180)

                    image270 = np.rot90(image180)
                    dataset_list.append(image270)

                    # append  lables 
                    label_list.append(classif[category])
                    # append also lables for rotated images
                    label_list.append(classif[category])
                    label_list.append(classif[category])
                    label_list.append(classif[category])


            

the path into the categories is:  C:\Users\sylvain\Documents\boulot\startup\radiology\ILDtempo\ild-cnn\patches\consolidation
the sub categories are :  ['apical', 'diffuse', 'non-relevant', 'peripheral_sub_pleural']
the path into the sub categories is: 
C:\Users\sylvain\Documents\boulot\startup\radiology\ILDtempo\ild-cnn\patches\consolidation\apical
the path into the sub categories is: 
C:\Users\sylvain\Documents\boulot\startup\radiology\ILDtempo\ild-cnn\patches\consolidation\diffuse
the path into the sub categories is: 
C:\Users\sylvain\Documents\boulot\startup\radiology\ILDtempo\ild-cnn\patches\consolidation\non-relevant
the path into the sub categories is: 
C:\Users\sylvain\Documents\boulot\startup\radiology\ILDtempo\ild-cnn\patches\consolidation\peripheral_sub_pleural
the path into the categories is:  C:\Users\sylvain\Documents\boulot\startup\radiology\ILDtempo\ild-cnn\patches\fibrosis
the sub categories are :  ['apical', 'basal', 'diffuse', 'non-relevant', 'perihilar', 'peripheral_

In [27]:
print (len(dataset_list),len(label_list))



(25572, 25572)


In [28]:
nb_elem = 25000 
indices = []  
resultatx=[]
resultaty= [] 
while nb_elem > 0:  
    i = random.randint(0, len(dataset_list) -1)  
    while i in indices: # tant que le tirage redonne un nombre déjà choisi  
        i = random.randint(0, len(dataset_list) -1)  
    indices.append(i)  
    nb_elem = nb_elem - 1  

for index in indices:  
    resultatx.append(dataset_list[index])  
    resultaty.append(label_list[index]) 

# transform dataset list into numpy array                   
X = np.array(resultatx)
#this is already in greyscale
# use only one of the 3 color channels as greyscale info
#X = dataset[:,:, :,1]

print 'dataset shape is now: ', X.shape

# 
y = np.array(resultaty)
# sampling item 22
#y[22]


dataset shape is now:  (25000L, 32L, 32L)


In [29]:
print (len(dataset_list),len(label_list))
print (len(resultatx),len(resultaty))

(25572, 25572)
(25000, 25000)


In [30]:
print X.shape
print y.shape

(25000L, 32L, 32L)
(25000L,)


In [31]:
X_train, X_intermediate, y_train, y_intermediate = train_test_split(X, y, test_size=0.5, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_intermediate, y_intermediate, test_size=0.5, random_state=42)
print X_train.shape
print X_val.shape
print X_test.shape
print y_train.shape
print y_test.shape
print y_val.shape
    

(12500L, 32L, 32L)
(6250L, 32L, 32L)
(6250L, 32L, 32L)
(12500L,)
(6250L,)
(6250L,)


In [32]:
# save the dataset and label set into serial formatted pkl 

pickle.dump(X_train, open( os.path.join(pickle_dir,"X_train.pkl"), "wb" ))
pickle.dump(X_test, open( os.path.join(pickle_dir,"X_test.pkl"), "wb" ))
pickle.dump(X_val, open(os.path.join(pickle_dir,"X_val.pkl"), "wb" ))
pickle.dump(y_train, open( os.path.join(pickle_dir,"y_train.pkl"), "wb" ))
pickle.dump(y_test, open( os.path.join(pickle_dir,"y_test.pkl"), "wb" ))
pickle.dump(y_val, open( os.path.join(pickle_dir,"y_val.pkl"), "wb" ))

In [33]:
# testing if pickls was working fine
recuperated_X_train = pickle.load( open( os.path.join(pickle_dir,"X_train.pkl"), "rb" ) )

In [34]:
recuperated_X_train

array([[[ 97, 100,  93, ...,  34,  36,  24],
        [ 74, 103, 106, ...,  53,  36,  40],
        [ 94,  93,  98, ...,  42,  52,  45],
        ..., 
        [ 12,  10,  24, ...,  18,  13,  21],
        [ 42,  22,  14, ...,  18,  22,  13],
        [ 46,  58,  36, ...,  14,   9,  16]],

       [[ 14,  11,  10, ...,  19,  36,  67],
        [ 16,  16,  17, ...,  23,  26,  26],
        [ 12,   8,   9, ...,   4,   1,   0],
        ..., 
        [  5,   2,   7, ...,   2,   1,   1],
        [  8,   6,  11, ...,  14,   5,  24],
        [  6,   4,   1, ...,  11,  66, 139]],

       [[108,  89,  35, ...,  21,  24,  22],
        [114,  93,  42, ...,  19,  32,   1],
        [106, 101,  51, ...,  34,  54,  19],
        ..., 
        [ 21,   1,   7, ...,  38,  15,   7],
        [ 51,  15,   8, ...,  24,  16,  14],
        [ 40,  39,  23, ...,  22,  23,  15]],

       ..., 
       [[  9,  10,  16, ...,  11,   6,  10],
        [ 21,  10,   7, ...,  20,  18,  11],
        [ 21,  15,   6, ...,   5,   4, 